## Lab Assignment 3
### Data Mining 7331 Section 403
---
- Brian Coari
- Stephen Merritt
- Cory Thigpen
- Quentin Thomas

## Business Understanding 
---
- Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs? (10 points)

## Data Understanding 
---
- Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems? (10 pts)
- Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs. (10 pts)

## Modeling and Evaluation
---
Different tasks will require different evaluation methods. Be as thorough as possible when analyzing the data you have chosen and use visualizations of the results to explain the performance and expected outcomes whenever possible. Guide the reader through your analysis with plenty of discussion of the results. Each option is broken down by:
- Train and adjust parameters (10 pts)
- Evaluate and Compare (10 pts)
- Visualize Results (10 pts)
- Summarize the Ramifications (20 pts)

Cluster Analysis
- Train: Perform cluster analysis using several clustering methods (adjust parameters).
- Eval: Use internal and/or external validation measures to describe and compare the clusterings and the clusters— how did you determine a suitable number of clusters for each method?
- Visualize: Use tables/visualization to discuss the found results. Explain each visualization in detail.
- Summarize: Describe your results. What findings are the most interesting and why?

## Deployment 
---
- Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling?  How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)?  How would your deploy your model for interested parties?  What other data should be collected?  How often would the model need to be updated, etc.? (10 pts)

## Exceptional Work 
---
- You have free reign to provide additional analyses or combine analyses. (10 pts)